<a href="https://colab.research.google.com/github/YemelongLafouetKevine/LAFOUET/blob/main/CMS.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [19]:

import pandas as pd
import zipfile
import os
# Unziping the uploaded dataset
zip_path = "hospitals_current_data.zip"   # already uploaded
extract_path = "hospital_data"

with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)

# list the extracted files to find the CSV name
os.listdir(extract_path)


['HCAHPS-State.csv',
 'Maternal_Health-Hospital.csv',
 'Hospital_General_Information.csv',
 'HOSPITAL_QUARTERLY_MSPB_6_DECIMALS.csv',
 'PCH_HCAHPS_STATE.csv',
 'FY2021_Distribution_of_Net_Change_in_Base_Op_DRG_Payment_Amt.csv',
 'Data_Updates_July_2025.csv',
 'PCH_Palliative_Care_NATIONAL.csv',
 'Healthcare_Associated_Infections-National.csv',
 'HOSPITAL_Data_Dictionary.pdf',
 'FY2021_Value_Based_Incentive_Payment_Amount.csv',
 'Outpatient_Imaging_Efficiency-State.csv',
 'OQR_OAS_CAHPS_STATE.csv',
 'OQR_OAS_CAHPS_BY_HOSPITAL.csv',
 'HCAHPS-Hospital.csv',
 'Medicare_Hospital_Spending_Per_Patient-Hospital.csv',
 'Outpatient_Imaging_Efficiency-Hospital.csv',
 'Outpatient_Imaging_Efficiency-National.csv',
 'Complications_and_Deaths-State.csv',
 'ASC_State.csv',
 'OAS_CAHPS_Footnotes.csv',
 'FY2021_Percent_Change_in_Medicare_Payments.csv',
 'Promoting_Interoperability-Hospital.csv',
 'IPFQR_QualityMeasures_Facility.csv',
 'HCAHPS-National.csv',
 'Healthcare_Associated_Infections-State.csv',

In [20]:
# Loading the dataset
df = pd.read_csv("hospital_data/Hospital_General_Information.csv")
print(df.shape)
df.head()

(5381, 38)


,Facility ID,Facility Name,Address,City/Town,State,ZIP Code,County/Parish,Telephone Number,Hospital Type,Hospital Ownership,...,Count of READM Measures Better,Count of READM Measures No Different,Count of READM Measures Worse,READM Group Footnote,Pt Exp Group Measure Count,Count of Facility Pt Exp Measures,Pt Exp Group Footnote,TE Group Measure Count,Count of Facility TE Measures,TE Group Footnote
0,010001,SOUTHEAST HEALTH MEDICAL CENTER,1108 ROSS CLARK CIRCLE,DOTHAN,AL,36301,HOUSTON,(334) 793-8701,Acute Care Hospitals,Government - Hospital District or Authority,...,0,11,0,NaN,8,8,NaN,12,11,NaN
1,010005,MARSHALL MEDICAL CENTERS,2505 U S HIGHWAY 431 NORTH,BOAZ,AL,35957,MARSHALL,(256) 593-8310,Acute Care Hospitals,Government - Hospital District or Authority,...,0,8,1,NaN,8,8,NaN,12,12,NaN
2,010006,NORTH ALABAMA MEDICAL CENTER,1701 VETERANS DRIVE,FLORENCE,AL,35630,LAUDERDALE,(256) 768-8400,Acute Care Hospitals,Proprietary,...,0,8,1,NaN,8,8,NaN,12,10,NaN
3,010007,MIZELL MEMORIAL HOSPITAL,702 N MAIN ST,OPP,AL,36467,COVINGTON,(334) 493-3541,Acute Care Hospitals,Voluntary non-profit - Private,...,0,7,0,NaN,8,8,NaN,12,7,NaN
4,010008,CRENSHAW COMMUNITY HOSPITAL,101 HOSPITAL CIRCLE,LUVERNE,AL,36049,CRENSHAW,(334) 335-3374,Acute Care Hospitals,Proprietary,...,0,2,0,NaN,8,Not Available,5.0,12,6,NaN


In [21]:
# Getting columns info
print(df.columns)
df.info()


Index(['Facility ID', 'Facility Name', 'Address', 'City/Town', 'State',
       'ZIP Code', 'County/Parish', 'Telephone Number', 'Hospital Type',
       'Hospital Ownership', 'Emergency Services',
       'Meets criteria for birthing friendly designation',
       'Hospital overall rating', 'Hospital overall rating footnote',
       'MORT Group Measure Count', 'Count of Facility MORT Measures',
       'Count of MORT Measures Better', 'Count of MORT Measures No Different',
       'Count of MORT Measures Worse', 'MORT Group Footnote',
       'Safety Group Measure Count', 'Count of Facility Safety Measures',
       'Count of Safety Measures Better',
       'Count of Safety Measures No Different',
       'Count of Safety Measures Worse', 'Safety Group Footnote',
       'READM Group Measure Count', 'Count of Facility READM Measures',
       'Count of READM Measures Better',
       'Count of READM Measures No Different', 'Count of READM Measures Worse',
       'READM Group Footnote', 'Pt Exp Gr

In [22]:
# Standardize column names
df.columns = df.columns.str.strip().str.lower().str.replace(' ', '_')


In [23]:
# Selecting useful policy and management columns
cols_to_keep = [
    'facility_id', 'facility_name', 'city/town', 'state', 'county/parish',
    'hospital_type', 'hospital_ownership', 'emergency_services',
    'meets_criteria_for_birthing_friendly_designation',
    'hospital_overall_rating'
]
df = df[cols_to_keep]


In [24]:
# Dropping duplicates and missing names
df = df.drop_duplicates(subset=['facility_name']).dropna(subset=['facility_name'])

# Converting hospital rating to numeric
df['hospital_overall_rating'] = pd.to_numeric(df['hospital_overall_rating'], errors='coerce')

# Converting emergency services to binary
df['emergency_services'] = df['emergency_services'].map({'Yes': 1, 'No': 0})

# Filling na
df['hospital_ownership'] = df['hospital_ownership'].fillna('Unknown')
df['hospital_type'] = df['hospital_type'].fillna('Unknown')

# Renaming columns for clarity
df = df.rename(columns={'city/town':'city', 'county/parish':'county'})


In [25]:
# Deriving region from state for Tableau filter
region_map = {
    'ME':'Northeast','NH':'Northeast','VT':'Northeast','MA':'Northeast','RI':'Northeast','CT':'Northeast',
    'NY':'Northeast','NJ':'Northeast','PA':'Northeast','OH':'Midwest','IN':'Midwest','IL':'Midwest',
    'MI':'Midwest','WI':'Midwest','MN':'Midwest','IA':'Midwest','MO':'Midwest','ND':'Midwest','SD':'Midwest',
    'NE':'Midwest','KS':'Midwest','DE':'South','MD':'South','DC':'South','VA':'South','WV':'South','KY':'South',
    'TN':'South','NC':'South','SC':'South','GA':'South','FL':'South','AL':'South','MS':'South','AR':'South',
    'LA':'South','OK':'South','TX':'South','MT':'West','ID':'West','WY':'West','CO':'West','NM':'West',
    'AZ':'West','UT':'West','NV':'West','CA':'West','OR':'West','WA':'West','AK':'West','HI':'West'
}
df['region'] = df['state'].map(region_map)


In [26]:
# Quick check
print("Average Rating by Ownership:")
print(df.groupby('hospital_ownership')['hospital_overall_rating'].mean().round(2).dropna())

print("\nEmergency Service Availability (%):")
print(df['emergency_services'].value_counts(normalize=True).mul(100).round(1))


Average Rating by Ownership:
hospital_ownership
Government - Federal                           3.22
Government - Hospital District or Authority    2.81
Government - Local                             2.71
Government - State                             2.78
Physician                                      3.33
Proprietary                                    2.64
Tribal                                         3.00
Veterans Health Administration                 4.20
Voluntary non-profit - Church                  3.23
Voluntary non-profit - Other                   3.18
Voluntary non-profit - Private                 3.18
Name: hospital_overall_rating, dtype: float64

Emergency Service Availability (%):
emergency_services
1    82.8
0    17.2
Name: proportion, dtype: float64


In [27]:
# Save the clean dataset as csv
df.to_csv("clean_hospital_data.csv", index=False)
print("Clean dataset saved as clean_hospital_data.csv")


Clean dataset saved as clean_hospital_data.csv
